# Feature Selection using BPSO for SDP

installations

In [1]:
 pip install pyswarms

  Stored in directory: C:\Users\afnan\AppData\Local\pip\Cache\wheels\8b\99\a0\81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


import libraries 

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import os
import pyswarms as ps

%load_ext autoreload
%autoreload 2
%matplotlib inline

### The methods 

Dataset prepration

In [3]:
def data_prepration(file):
    df=pd.read_csv(file)
    if( '-' in file):
        X=np.array(df.drop(['bug','name.1','name', 'version'], axis=1))
    else:
        X=np.array(df.drop('bug', axis=1))
    y=np.array(df.bug)
    
    return X,y

plot data per feature (each class with different color)

In [4]:
def plot_features(features,labels):
    df = pd.DataFrame(features)
    df['labels'] = pd.Series(labels)

    sns.pairplot(df, hue='labels' , diag_kind='hist')

fitness function for each particle

In [16]:
def f_per_particle(m):
    classifier = KNeighborsClassifier(n_neighbors=5)
    """Computes for the fitness function per particle

    Inputs
    ------
    m : numpy.ndarray
        Binary mask that can be obtained from BinaryPSO, will
        be used to mask features.

    Returns
    -------
    numpy.ndarray
        Computed finess function
    """
    
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = X_train
        X_subset_test= X_test
    else:
        X_subset = X_train[:,m==1]
        X_subset_test= X_test[:,m==1]
        
    
    # Perform classification and compute the fitness function
    classifier.fit(X_subset, y_train)
    j = (classifier.predict(X_subset_test) != y_test).mean() + np.count_nonzero(m)
#     j=np.count_nonzero(m)

    return j

Define fitness function for whole swarm

In [11]:
def f(x):
    """Higher-level method to do classification in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [f_per_particle(x[i]) for i in range(n_particles)]
    
    return np.array(j)

select the features from the set of features X using BPSO on the specfied parameters

In [17]:
def select_features(nparticles):
  
    total_features = X.shape[1]
    
    dimensions = total_features
    
    optimizer = ps.discrete.BinaryPSO(n_particles=nparticles, dimensions=dimensions, options=options)
    
    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=iters)
    
    ds=file.split("\\")[3][0:-4]
    print("cost: "+str(cost))
    fr.write(clas+"-"+ds+" dataset:\ncost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    return cost, pos

Train clssifier on the selected features (corrposining to pos which =1)

In [23]:
def train_and_test(clas,X_train,X_test,y_train,y_test,pos):   
    
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='Linear Regression'):
        classfier = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
        
    # Get the selected features from the final positions
    X_selected_features = X_train[:,pos==1]  # subset

    # Perform classification and store performance in P
    classifier.fit(X_selected_features,y_train)
    selected_test= X_test[:,pos==1]

    # Compute performance
    subset_performance = (classifier.predict(selected_test) == y_test).mean()
    fr.write("\nsubset_performance:"+str(subset_performance)+"\n")
    
    print('Subset performance: %.3f' % (subset_performance))

## The main cell 

here the classifier is created, options, number of iterations are set, and other methods are called

each results for each dataset are stored in results folder with file of the classifier name

In [21]:
import time
from sklearn.model_selection import train_test_split

# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# clas='NB'

# from sklearn import svm
# classifier = svm.SVC(kernel='linear')
# clas='SVM'

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
clas='KNN'

# from sklearn import linear_model
# classifier = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
# clas='Linear Regression'

options = {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k':40, 'p':2}
# k : number of neighbors to be considered. Must be a positive integer less than n_partcles 
# p: the Minkowski p-norm to use. 1 is the sum-of-absolute values (or L1 distance) while 2 is the Euclidean (or L2) distance
iters= 1000

fr = open("Results/"+clas+"2.txt", "a")
fr.write(clas+'\nparameters: '+str(options)+"\n")
root_folder="D:\\SDP\\Dataset Dr.Sultan"
files = [os.path.join(root_folder, x) for x in os.listdir(root_folder)]
for file in files[0:1] :
    start= time.time()
    X,y = data_prepration(file)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    cost, pos,selectedX,selectedy = select_features(40 )
    train_and_test(clas,X_train,X_test,y_train,y_test,pos)
    period= time.time() - start 
    fr.write('FS, training, and testing time= '+str(period)+" seconds\n")
fr.close()

2021-11-29 19:22:34,534 - pyswarms.discrete.binary - INFO - Optimize for 1000 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k': 40, 'p': 2}

pyswarms.discrete.binary:   0%|                                                                                 |0/1000
pyswarms.discrete.binary:   0%|                                                                 |0/1000, best_cost=6.23
pyswarms.discrete.binary:   0%|                                                                 |1/1000, best_cost=6.23
pyswarms.discrete.binary:   0%|                                                                 |1/1000, best_cost=6.23
pyswarms.discrete.binary:   0%|▏                                                                |2/1000, best_cost=6.23
pyswarms.discrete.binary:   0%|▏                                                                |2/1000, best_cost=6.23
pyswarms.discrete.binary:   0%|▏                                                                |3/1000, best_cost=6.23
pyswarms.discrete.bi

pyswarms.discrete.binary:   3%|██                                                              |32/1000, best_cost=4.22
pyswarms.discrete.binary:   3%|██                                                              |33/1000, best_cost=4.22
pyswarms.discrete.binary:   3%|██                                                              |33/1000, best_cost=4.22
pyswarms.discrete.binary:   3%|██▏                                                             |34/1000, best_cost=4.22
pyswarms.discrete.binary:   3%|██▏                                                             |34/1000, best_cost=4.22
pyswarms.discrete.binary:   4%|██▏                                                             |35/1000, best_cost=4.22
pyswarms.discrete.binary:   4%|██▏                                                             |35/1000, best_cost=4.22
pyswarms.discrete.binary:   4%|██▎                                                             |36/1000, best_cost=4.22
pyswarms.discrete.binary:   4%|██▎      

pyswarms.discrete.binary:   6%|████▏                                                           |65/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▏                                                           |66/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▏                                                           |66/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▎                                                           |67/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▎                                                           |67/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▎                                                           |68/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▎                                                           |68/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▍                                                           |69/1000, best_cost=4.21
pyswarms.discrete.binary:   7%|████▍    

pyswarms.discrete.binary:  10%|██████▎                                                         |98/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▎                                                         |99/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▎                                                         |99/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▎                                                        |100/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▎                                                        |100/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▎                                                        |101/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▎                                                        |101/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▍                                                        |102/1000, best_cost=3.22
pyswarms.discrete.binary:  10%|██████▍  

pyswarms.discrete.binary:  13%|████████▎                                                      |131/1000, best_cost=3.22
pyswarms.discrete.binary:  13%|████████▎                                                      |132/1000, best_cost=3.22
pyswarms.discrete.binary:  13%|████████▎                                                      |132/1000, best_cost=3.22
pyswarms.discrete.binary:  13%|████████▍                                                      |133/1000, best_cost=3.22
pyswarms.discrete.binary:  13%|████████▍                                                      |133/1000, best_cost=3.22
pyswarms.discrete.binary:  13%|████████▍                                                      |134/1000, best_cost=3.22
pyswarms.discrete.binary:  13%|████████▍                                                      |134/1000, best_cost=3.22
pyswarms.discrete.binary:  14%|████████▌                                                      |135/1000, best_cost=3.22
pyswarms.discrete.binary:  14%|████████▌

...

pyswarms.discrete.binary:  92%|██████████████████████████████████████████████████████████▏    |923/1000, best_cost=2.19
pyswarms.discrete.binary:  92%|██████████████████████████████████████████████████████████▏    |924/1000, best_cost=2.19
pyswarms.discrete.binary:  92%|██████████████████████████████████████████████████████████▏    |924/1000, best_cost=2.19
pyswarms.discrete.binary:  92%|██████████████████████████████████████████████████████████▎    |925/1000, best_cost=2.19
pyswarms.discrete.binary:  92%|██████████████████████████████████████████████████████████▎    |925/1000, best_cost=2.19
pyswarms.discrete.binary:  93%|██████████████████████████████████████████████████████████▎    |926/1000, best_cost=2.19
pyswarms.discrete.binary:  93%|██████████████████████████████████████████████████████████▎    |926/1000, best_cost=2.19
pyswarms.discrete.binary:  93%|██████████████████████████████████████████████████████████▍    |927/1000, best_cost=2.19
pyswarms.discrete.binary:  93%|█████████

pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▏  |956/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▎  |957/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▎  |957/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▎  |958/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▎  |958/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▍  |959/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▍  |959/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|████████████████████████████████████████████████████████████▍  |960/1000, best_cost=2.19
pyswarms.discrete.binary:  96%|█████████

pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▎|989/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▎|990/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▎|990/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▍|991/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▍|991/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▍|992/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▍|992/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|██████████████████████████████████████████████████████████████▌|993/1000, best_cost=2.19
pyswarms.discrete.binary:  99%|█████████

cost: 2.186991869918699


ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
pos=[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]

In [27]:
 train_and_test(clas,X_train,X_test,y_train,y_test,pos)

Subset performance: 0.760
